
The dataset of ancient Eurasians (see [DataS1](https://github.com/sarabehnamian/Origins-of-Ancient-Eurasian-Genomes/tree/main/steps/Step%200)). The dataset contains **genetic data** of **ancient Eurasians** from **different time periods**. The data is in the form of **SNP patterns**. The **SNP patterns** are **haplotypes** of **individuals**. Each **haplotype** is a **string** of **0s** and **1s**. The **length** of the **haplotype** is the **number of SNPs**.


Develop an interface that allows selecting **populations** and **time** and calculates the **minor allele frequencies** of the chosen populations. The program will **merge patterns** that are too close. The results will be printed on the screen as **admixture plots**.

# PLINK 1.9 

[PLINK](https://www.cog-genomics.org/plink/1.9/) is a free, open-source whole genome association analysis toolset, designed to perform a range of basic, large-scale analyses in a computationally efficient manner. The focus of PLINK is purely on analysis of genotype/phenotype data, so there is no support for a file format for storing sequence data or working with the sequence data directly.

[_More..._](https://zzz.bwh.harvard.edu/plink/)

[_Tutorial_](https://zzz.bwh.harvard.edu/plink/tutorial.shtml)

**_Installation_**

` conda install -c bioconda plink `

